In [1]:
import zipfile
import os

# Update the file path to the location where you have the zip file
zip_file_path = r'C:\Users\srija\Downloads\bank+marketing (1).zip'
extraction_directory = r'C:\Users\srija\Downloads\bank_marketing_data\bank-additional\bank-additional'

# Create the extraction directory if it doesn't exist
os.makedirs(extraction_directory, exist_ok=True)

# Extract the zip file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extraction_directory)

# List the extracted files
extracted_files = os.listdir(extraction_directory)
print("Extracted files:", extracted_files)

Extracted files: ['bank-additional.zip', 'bank.zip']


In [3]:
import pandas as pd

# Path to the extracted CSV file
csv_file_path = os.path.join('bank-additional-full.csv')

# Load the dataset
df = pd.read_csv(csv_file_path, delimiter=';')

# Display the first few rows of the dataset
print(df.head())
df = pd.read_csv(csv_file_path, sep=';')

   age        job  marital    education  default housing loan    contact  \
0   56  housemaid  married     basic.4y       no      no   no  telephone   
1   57   services  married  high.school  unknown      no   no  telephone   
2   37   services  married  high.school       no     yes   no  telephone   
3   40     admin.  married     basic.6y       no      no   no  telephone   
4   56   services  married  high.school       no      no  yes  telephone   

  month day_of_week  ...  campaign  pdays  previous     poutcome emp.var.rate  \
0   may         mon  ...         1    999         0  nonexistent          1.1   
1   may         mon  ...         1    999         0  nonexistent          1.1   
2   may         mon  ...         1    999         0  nonexistent          1.1   
3   may         mon  ...         1    999         0  nonexistent          1.1   
4   may         mon  ...         1    999         0  nonexistent          1.1   

   cons.price.idx  cons.conf.idx  euribor3m  nr.employed

In [5]:
print(df.isnull().sum())

# Get an overview of the dataset
print(df.info())

# Display basic statistics of the dataset
print(df.describe())

age               0
job               0
marital           0
education         0
default           0
housing           0
loan              0
contact           0
month             0
day_of_week       0
duration          0
campaign          0
pdays             0
previous          0
poutcome          0
emp.var.rate      0
cons.price.idx    0
cons.conf.idx     0
euribor3m         0
nr.employed       0
y                 0
dtype: int64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 21 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   age             41188 non-null  int64  
 1   job             41188 non-null  object 
 2   marital         41188 non-null  object 
 3   education       41188 non-null  object 
 4   default         41188 non-null  object 
 5   housing         41188 non-null  object 
 6   loan            41188 non-null  object 
 7   contact         41188 non-null  object 
 8   mon

In [7]:
from sklearn.preprocessing import LabelEncoder

# Encode the target variable ('y')
label_encoder = LabelEncoder()
df['y'] = label_encoder.fit_transform(df['y'])  # 'yes' becomes 1 and 'no' becomes 0

In [9]:
# Identify categorical columns
categorical_columns = df.select_dtypes(include=['object']).columns

# One-hot encode the categorical features
df_encoded = pd.get_dummies(df, columns=categorical_columns, drop_first=True)

In [11]:
from sklearn.model_selection import train_test_split

# Features (X) and target (y)
X = df_encoded.drop('y', axis=1)
y = df_encoded['y']

# Split the data into training and testing sets (80% training, 20% testing)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [13]:
from sklearn.tree import DecisionTreeClassifier

# Initialize the classifier
clf = DecisionTreeClassifier(random_state=42)

# Train the classifier
clf.fit(X_train, y_train)

DecisionTreeClassifier(random_state=42)

In [15]:
# Predict the target values for the test set
y_pred = clf.predict(X_test)

In [17]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy * 100:.2f}%")

# Detailed classification report
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

# Confusion matrix
print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred))

Accuracy: 88.69%

Classification Report:
              precision    recall  f1-score   support

           0       0.94      0.93      0.94      7303
           1       0.50      0.52      0.51       935

    accuracy                           0.89      8238
   macro avg       0.72      0.73      0.72      8238
weighted avg       0.89      0.89      0.89      8238


Confusion Matrix:
[[6817  486]
 [ 446  489]]


In [19]:
from sklearn.tree import export_graphviz
import graphviz

# Export the decision tree to a DOT file
dot_data = export_graphviz(clf, out_file=None, 
                           feature_names=X.columns,  
                           class_names=['No', 'Yes'],  
                           filled=True, rounded=True,  
                           special_characters=True)  

# Visualize the decision tree
graph = graphviz.Source(dot_data)  
graph.render("decision_tree")  # This saves the visualization as 'decision_tree.pdf'
graph.view()

'decision_tree.pdf'